In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Regularization
[name for name in dir(keras.regularizers) if not name.startswith('_')]

['L1',
 'L1L2',
 'L2',
 'OrthogonalRegularizer',
 'Regularizer',
 'deserialize',
 'get',
 'l1',
 'l1_l2',
 'l2',
 'orthogonal_regularizer',
 'serialize']

In [4]:
r_l1 = keras.regularizers.l1(l1=0.01)
r_l2 = keras.regularizers.l2(l2=0.02)
r_l1_l2 = keras.regularizers.l1_l2(l1=0.01, l2=0.01)

In [5]:
#load data
from tensorflow.keras.datasets import fashion_mnist
(x_train_set, y_train_set), (x_test, y_test) = fashion_mnist.load_data()

#Split data
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train_set, y_train_set, random_state=1)

#Preprocessing
pixel_means = x_train.mean(axis=0, keepdims=True)
pixel_stds = x_train.std(axis=0, keepdims=True)
x_train_scaled = (x_train - pixel_means) / pixel_stds
x_valid_scaled = (x_valid - pixel_means) / pixel_stds
x_test_scaled = (x_test - pixel_means) / pixel_stds

In [6]:
tf.keras.backend.clear_session()
np.random.seed(1)
tf.random.set_seed(1)

In [9]:
r_l2 = keras.regularizers.l2(l2=0.01)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal', kernel_regularizer=r_l2),
    keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal', kernel_regularizer=r_l2),
    keras.layers.Dense(10,activation='softmax', kernel_regularizer=r_l2)
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [12]:
train = model.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1407/1407 [==============================] - 13s 9ms/step - loss: 1.6802 - accuracy: 0.8148 - val_loss: 0.8575 - val_accuracy: 0.7857
Epoch 2/2
1407/1407 [==============================] - 13s 9ms/step - loss: 0.7362 - accuracy: 0.8286 - val_loss: 0.7210 - val_accuracy: 0.8303


In [13]:
#Dropout

model2 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation='relu', kernel_initializer='he_normal'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10,activation='softmax')
])

In [14]:
model2.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [15]:
train2 = model2.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1407/1407 [==============================] - 13s 8ms/step - loss: 0.5746 - accuracy: 0.7939 - val_loss: 0.4080 - val_accuracy: 0.8485
Epoch 2/2
1407/1407 [==============================] - 13s 9ms/step - loss: 0.4409 - accuracy: 0.8386 - val_loss: 0.3660 - val_accuracy: 0.8676


In [16]:
model2.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3892 - accuracy: 0.8545


[0.3892241418361664, 0.8544999957084656]

In [17]:
y_proba = model.predict(x_test_scaled)
y_proba[:3].round(2)

313/313 [==============================] - 1s 1ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.06, 0.  , 0.33, 0.  , 0.61],
       [0.  , 0.  , 0.86, 0.  , 0.03, 0.  , 0.1 , 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

In [19]:
y_pred = np.argmax(y_proba,axis=1)
y_pred[:3]

array([9, 2, 1], dtype=int64)

In [20]:
y_test[:3]

array([9, 2, 1], dtype=uint8)

In [24]:
#MC Dropout

y_probs = np.stack([model2(x_test_scaled[:3], training=True) for _ in range(100)])
y_probs.shape

(100, 3, 10)

In [25]:
y_proba_mc = y_probs.mean(axis=0)
np.round(y_proba_mc,2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.15, 0.  , 0.8 ],
       [0.  , 0.  , 0.96, 0.  , 0.02, 0.  , 0.02, 0.  , 0.  , 0.  ],
       [0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

In [26]:
y_pred = np.argmax(y_proba_mc, axis=1)
y_pred

array([9, 2, 1], dtype=int64)

In [30]:
#Alpha Dropout

model3 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation='selu', kernel_initializer='lecun_normal'),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10,activation='softmax')
])

In [34]:
model3.compile(loss='sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])

In [35]:
train3 = model3.fit(x_train_scaled, y_train, epochs=2, validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1407/1407 [==============================] - 13s 9ms/step - loss: 0.6664 - accuracy: 0.7630 - val_loss: 0.6163 - val_accuracy: 0.8271
Epoch 2/2
1407/1407 [==============================] - 14s 10ms/step - loss: 0.5077 - accuracy: 0.8118 - val_loss: 0.4989 - val_accuracy: 0.8473


In [36]:
#Max-Norm Regularization

model4 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation='selu', 
                       kernel_initializer='lecun_normal',
                      kernel_constraint = keras.constraints.max_norm(1.)),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation='selu', 
                       kernel_initializer='lecun_normal',
                      kernel_constraint = keras.constraints.max_norm(1.)),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10,activation='softmax')
])

In [37]:
model4.compile(loss='sparse_categorical_crossentropy',
              optimizer = 'nadam',
              metrics=['accuracy'])

In [38]:
train4 = model4.fit(x_train_scaled, y_train, epochs=2,
                   validation_data=(x_valid_scaled, y_valid))

Epoch 1/2
1407/1407 [==============================] - 16s 11ms/step - loss: 0.6682 - accuracy: 0.7628 - val_loss: 0.6030 - val_accuracy: 0.8254
Epoch 2/2
1407/1407 [==============================] - 16s 11ms/step - loss: 0.5185 - accuracy: 0.8097 - val_loss: 0.5257 - val_accuracy: 0.8401
